In [41]:
import pandas as pd
import numpy as np
from nltk.stem.snowball import SnowballStemmer
from sklearn import tree

In [26]:
df_train = pd.read_csv("../input/train.csv", encoding="ISO-8859-1")
df_test = pd.read_csv('../input/test.csv', encoding="ISO-8859-1")
df_attr = pd.read_csv('../input/attributes.csv')
df_prod_desc = pd.read_csv('../input/product_descriptions.csv', index_col="product_uid")

In [61]:
print df_test.head()
print "\n\n"
print df_train.head()

   id  product_uid                         product_title  \
0   1       100001   "Simpson Strong-Tie 12-Gauge Angle"   
1   4       100001     Simpson Strong-Tie 12-Gauge Angle   
2   5       100001     Simpson Strong-Tie 12-Gauge Angle   
3   6       100001     Simpson Strong-Tie 12-Gauge Angle   
4   7       100001     Simpson Strong-Tie 12-Gauge Angle   

                 search_term  
0        "90 degree bracket"  
1           metal l brackets  
2           simpson sku able  
3       simpson strong  ties  
4  simpson strong tie hcc668  



   id  product_uid                                      product_title  \
0   2       100001                "Simpson Strong-Tie 12-Gauge Angle"   
1   3       100001                "Simpson Strong-Tie 12-Gauge Angle"   
2   9       100002  BEHR Premium Textured DeckOver 1-gal. #SC-141 ...   
3  16       100005  Delta Vero 1-Handle Shower Only Faucet Trim Ki...   
4  17       100005  Delta Vero 1-Handle Shower Only Faucet Trim Ki...   

          s

In [38]:
train_num_feat = pd.DataFrame(0, index=np.arange(len(df_train)), columns=["search_in_title",\
                                     "search_in_desc"])
test_num_feat = pd.DataFrame(0, index=np.arange(len(df_test)), columns=["search_in_title",\
                                     "search_in_desc"])

# Count the training data occurrences
for i in range(0, df_train.shape[0]):
    search = df_train["search_term"][i]  # Search term used by user
    title = df_train["product_title"][i] # Product Title
    product_id = df_train.iloc[[i]]["product_uid"][i]  # Get product_uid of current index in df_train
    description = df_prod_desc.loc[product_id]["product_description"] # Description of product id

    lower_title = title.lower()
    lower_description = description.lower()
    
    for word in search.split():
        lower_word = word.lower()
        
        if lower_word in lower_title:
            train_num_feat["search_in_title"][i] += 1
            
        if lower_word in lower_description:
            train_num_feat["search_in_desc"][i] += 1
            
# Count the testing data occurrences
for i in range(0, df_test.shape[0]):
    search = df_test["search_term"][i]  # Search term used by user
    title = df_test["product_title"][i] # Product Title
    product_id = df_test.iloc[[i]]["product_uid"][i]  # Get product_uid of current index in df_train
    description = df_prod_desc.loc[product_id]["product_description"] # Description of product id

    lower_title = title.lower()
    lower_description = description.lower()
    
    for word in search.split():
        lower_word = word.lower()

        if lower_word in lower_title:
            test_num_feat["search_in_title"][i] += 1
            
        if lower_word in lower_description:
            test_num_feat["search_in_desc"][i] += 1

In [62]:
print train_num_feat.head(10)

   search_in_title  search_in_desc
0                1               1
1                0               0
2                2               2
3                1               1
4                3               3
5                1               2
6                2               2
7                0               0
8                2               2
9                2               2


In [57]:
clf = tree.DecisionTreeRegressor()
clf.fit(train_num_feat, df_train["relevance"])
yhat_val = clf.predict(test_num_feat)
df_yhat = pd.DataFrame(yhat_val, columns=["relevance"], index=df_test["id"])
df_yhat.to_csv('yhat.csv')
print yhat.head()

    relevance
id           
1    2.115264
4    2.366940
5    2.366940
6    2.385426
7    2.527643
